### Try CNN to classficaton

In [6]:
import math
import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt # plt 用于显示图片
import matplotlib.image as mpimg # mpimg 用于读取图片
from torch.autograd import Variable
from PIL import Image

In [7]:
def ImageToTensor(path):
    RGBimg = np.array(Image.open(path).convert("RGB"))
    # print(RGBimg.shape)
    RGBimg = np.transpose(RGBimg, (2, 0, 1)) # 将通道放到第一维
    # print(RGBimg.shape)
    torchImg = torch.from_numpy(RGBimg)
    # print(torchImg.shape)
    torchImg = Variable(torch.unsqueeze(torchImg, dim=0).float(), requires_grad=False)
    # print(torchImg.shape)
    return torchImg

In [24]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet,self).__init__()
        self.model = torchvision.models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(2048, 10)

        pretrained_dict = torch.load('C:/Users/Lenovo/Desktop/ResNet_no_adaptation.pth', map_location='cpu')
        model_dict = self.model.state_dict()

        for k in model_dict.keys():
            if(('module.'+k) in pretrained_dict.keys()):
                model_dict[k]=pretrained_dict.get(('module.'+k))
        self.model.load_state_dict(model_dict)
    def forward(self,x):
        out=self.model(x)
        return out

In [25]:
net = ResNet()
# print(net)
images = ImageToTensor('./pkq.png')
outputs = net(images)
print(outputs)

tensor([[-0.0547,  0.0501,  0.2295,  0.2551, -0.5024,  0.0009,  0.2564,  0.1411,
         -0.1123, -0.3501]], grad_fn=<AddmmBackward>)


In [27]:
def test(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())

In [ ]:
def criterion

In [29]:
criterion = nn.CrossEntropyLoss() # 使用CrossEntropyLoss损失
optm = torch.optim.Adam(net.parameters()) # Adam优化
epochs = 1000 # 训练1000次
train_data = []
train_lab = []
test_data = []
test_lab = []

In [ ]:
for i in range(epochs):
    # 指定模型为训练模式，计算梯度
    net.train()
    # 输入值都需要转化成torch的Tensor
    x=torch.from_numpy(train_data).float()
    y=torch.from_numpy(train_lab).long()
    y_hat=net(x)
    loss=criterion(y_hat,y) # 计算损失
    optm.zero_grad() # 前一步的损失清零
    loss.backward() # 反向传播
    optm.step() # 优化
    if (i+1)%100 ==0 : # 这里我们每100次输出相关的信息
        # 指定模型为计算模式
        net.eval()
        test_in=torch.from_numpy(test_data).float()
        test_l=torch.from_numpy(test_lab).long()
        test_out=net(test_in)
        # 使用我们的测试函数计算准确率
        accu=test(test_out,test_l)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.4f}".format(i+1,loss.item(),accu))
